In [37]:
import requests
import pandas as pd

In [38]:
#API Banxico
banxico_key = "85c8edeab72128102cdad764f0a2d4532c336d22690a51104b500fb6f077ed4c"

In [39]:
url = "https://www.banxico.org.mx/SieAPIRest/service/v1/series/"

In [75]:
#Series
# Serie de fondeo SF283
Series_fondeo = 'SF283'
#Total series prestamos
Series_prestamos = "SF235757"

In [76]:
#Fechas AAAA-MM-dd
#fecha_inicial = input(print("Fecha inicial consulta AAAA-MM-dd"))
fecha_inicial = "1999-01-01"
fecha_final = "2019-12-31"
#fecha_final = input(print("Fecha final consulta AAAA-MM-dd"))

In [123]:
#DAto Fondeo
url_consulta_tasa = f"{url}{Series_fondeo}/datos/{fecha_inicial}/{fecha_final}?token={banxico_key}"
data = requests.get(url_consulta).json()
fondeo_data = data['bmx']['series'][0]['datos']
fondeo_data_df = pd.DataFrame(fondeo_data)
fondeo_data_df.rename(columns = {'fecha' : 'date', 'dato':'policy_rate'}, inplace = True)
#fondeo_data_df.set_index('id', inplace = True)
fondeo_data_df.tail()


,date,policy_rate
247,01/08/2019,8.37
248,01/09/2019,8.20
249,01/10/2019,8.00
250,01/11/2019,7.87
251,01/12/2019,7.70


In [124]:
#Dato Prestamo
url_consulta_loans = f"{url}{Series_prestamos}/datos/{fecha_inicial}/{fecha_final}?token={banxico_key}"
data_loans = requests.get(url_consulta_loans).json()
loans_data = data_loans['bmx']['series'][0]['datos']
loans_data_df = pd.DataFrame(loans_data)
loans_data_df.rename(columns = {'fecha' : 'date', 'dato':'total_loans'}, inplace = True)
#loans_data_df.set_index('id', inplace = True)
loans_data_df.tail()


,date,total_loans
247,01/08/2019,"4,688,826,865.00"
248,01/09/2019,"4,663,167,704.00"
249,01/10/2019,"4,641,863,408.00"
250,01/11/2019,"4,713,995,432.00"
251,01/12/2019,"4,672,516,126.00"


In [125]:
from sqlalchemy import create_engine
rds_connection_string = "postgres:<password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [126]:
engine.table_names()

[]

In [127]:
loans_data_df.to_sql(name = 'rates', con = engine, if_exists = 'append', index = True)
fondeo_data_df.to_sql(name = 'loans', con = engine, if_exists = 'append', index = True)